need to do - review colours for red, green and yellow
improve x-y coordinate stuff
work on get force stuff!

For some reason my shit is not working - might have to try simple approach of just going over it?

get rotate going!!

Get funny eyes!

In [23]:
# Import all libraries!
import pyttsx
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np
import requests
import shutil
import time
from urx.robotiq_two_finger_gripper import Robotiq_Two_Finger_Gripper
import logging
import urx
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
# from urx.urrtmon import URRTMonitor



cam_init_pos = [-0.00043183961977177887, -1.651595417653219, -1.226389233266012, -1.276344124470846, -4.72298783460726, 0.001785649568773806]
bucket_pos = [1.6486937999725342, -2.4073990027057093, -1.8858941237079065, -0.36886865297426397, -4.742553655301229, 0.0471811443567276]
hardcode_object_loc = [-0.024045769368306935, -1.8617809454547327, -1.511141602193014, -1.4511950651751917, -4.672266546879904, 0.047373268753290176]
bucket_pick = [-0.04573041597475225, -1.8159015814410608, -1.2815788427936, -1.4384973684894007, -4.67054266134371, 0.04719312861561775]

high_point = [0.90383380651474, -1.4072287718402308, -0.237281624470846, -0.17934972444643194, -4.722544852887289, -0.05534726778139287]

In [8]:

test_robot(a = 1, v = 1)
t = 0
while t < 10:
    try:
        rob = urx.Robot("192.168.1.6", use_rt=True)
    except:
        print("trying to connect again")
        t += 1
    else:
        break

a = 0.2; v = 0.05

rob.translate((0, 0, -0.4), acc=a, vel=v, wait=False)
t = 0
while True:
    f = rob.get_tcp_force()
    print(f)
    print(f[2])
    if f[2] > 20:
        rob.translate((0, 0, 0.03), acc=a, vel=v, wait=False)
        break
    time.sleep(0.01)
#     t+=1

print("found bottom!")
rob.close()

('robot tcp is at pose: ', [-0.0004556814776819351, -1.6516073385821741, -1.2263172308551233, -1.2765000502215784, -4.72296387354006, 0.001785649568773806])
[-3.41292087 -3.18311934 -3.79686491  1.92359252  0.19007476 -0.64249825]
-3.796864905001801
[-3.74699327 -2.94311413 -4.54109623  1.75239489 -0.09865735 -0.67632356]
-4.5410962338806975
[-3.80832048 -3.30103636 -4.41626889  1.84115751 -0.01166106 -0.70819127]
-4.416268886798493
[-4.19219203 -2.89731379 -4.81167527  1.939272   -0.01981155 -0.76732203]
-4.811675268542481
[-3.66690245 -3.70658585 -3.65328135  1.81205879  0.04553782 -0.62396602]
-3.6532813538090534
[-1.78684188 -3.47697041 -0.79876609  2.00618197  0.73968744 -0.61588636]
-0.7987660883360665
[-10.74501873  -0.98608171 -16.1400348    1.6326729   -2.51411502
  -0.83679153]
-16.140034796889257
[ -6.230939    -2.56925612 -11.30897233   1.94461068  -0.89254207
  -0.69956672]
-11.308972328117584
[-2.41020829 -3.89861624 -6.74803977  1.93038328  0.07100212 -0.62187561]
-6.748

[ 9.40333923  0.71230143  4.24713495  1.05956524  1.35154092 -0.53772115]
4.247134954568678
[ 8.6471395   3.05403458  7.54440394 -0.61501767  2.89145402 -1.53797149]
7.5444039353368515
[ 7.01900772  4.20084024  0.76533907 -1.08363259 -0.51145906 -1.79830871]
0.7653390664008242
[ 3.33349386  4.78424582  2.88234939 -1.52093257  1.44874882 -1.84172277]
2.8823493909114957
[ 3.85263878  4.288383    3.39111979 -0.79490781  0.48233186 -1.37426323]
3.391119786482636
[ 3.19775872  4.40414515  6.41888516 -1.27205645  0.92007839 -1.50680249]
6.418885155974229
[-0.90518962  4.17625467 13.2199587  -1.28702314  3.23348782 -1.49910187]
13.21995870113615
[-5.58751223  1.80142148  2.92222311  1.59856767 -1.69998309  0.44680812]
2.92222310964617
[-11.11722976   0.14962266   5.13500133   3.05279938   0.71306224
   1.40798908]
5.135001334944622
[-6.27795946  0.23456321 10.91623198  1.98557877  0.81449539  0.87651472]
10.91623198246886
[ 0.19150342 -3.02289525 16.66790157  2.63951161  0.25264598  1.3811586

Exception in thread Thread-11:
Traceback (most recent call last):
  File "C:\Users\61457\Anaconda3\envs\p2\lib\threading.py", line 801, in __bootstrap_inner
    self.run()
  File "C:\Users\61457\Anaconda3\envs\p2\lib\site-packages\urx-0.11.0-py2.7.egg\urx\ursecmon.py", line 289, in run
    data = self._get_data()
  File "C:\Users\61457\Anaconda3\envs\p2\lib\site-packages\urx-0.11.0-py2.7.egg\urx\ursecmon.py", line 336, in _get_data
    tmp = self._s_secondary.recv(1024)
timeout: timed out



In [ ]:
%matplotlib inline


# initalise
rob = rob_connect()
robotiqgrip = Robotiq_Two_Finger_Gripper(rob)
a = 0.7; v = 2

engine.say('Hello, Welcome to Program 1, I will now move into my initial position')
engine.runAndWait()

robotiqgrip.open_gripper()
go_to_start(rob)

engine.say('Please type in the number of bins you would like to use. Next enter the colour for each after you have moved the robot arm')
engine.runAndWait()

[colours, bins] = teach_bins(rob)
i = 0

while True:
    if i == len(colours):
        print("finishing program")
        engine.say('The first part of the program has finished, as there are no more coloured balls on the table')
        engine.runAndWait()
        break
        
    go_to_start(rob)
    time.sleep(1)
    get_image()
    distance = detect_photos(colours[i])
    
    if distance == None:
        i+=1
        continue
        

    [x_dist, y_dist] = convert_to_mm(distance[0], distance[1])
    
    engine.say('Watch me pick up a ' + colours[i] 'ball')
    engine.runAndWait()
    
    rob.translate((y_dist-0.095, x_dist, -0.33), acc=a, vel=v)

    robotiqgrip.close_gripper()
    rob.movejs([cam_init_pos, bins[i]], acc = a, vel = v, radius=0.2)
    robotiqgrip.open_gripper()
    
    engine.say('Kobeeeeee')
    engine.runAndWait()

    
    
### SEARCHING THE AREA

engine.say('Time to scan the area for more balls')
engine.runAndWait()


angle = 45
while angle <= 315:
    try:
        
        pose = rob.getj()
        pose[0] = pose[0] + 0.785398
        rob.movej(pose, acc = a, vel = v)
        
        print("current angle is: " + str(angle))
        engine.say('The current angle is' + str(angle))
        engine.runAndWait()
        
        
        time.sleep(2)
        get_image()
        distance = detect_photos('green')

    #     change_coordinates(0.1, 0.05, angle)

        if distance == None:
            engine.say('Nothing here')
            engine.runAndWait()
            
            angle += 45
            continue
        
        engine.say('Oooh, Found something')
        engine.runAndWait()
        
        [x_dist, y_dist] = convert_to_mm(distance[0], distance[1])
    #     rob.translate((y_dist-0.095, x_dist, -0.33), acc=a, vel=v)
        print(x_dist, y_dist)
        change_coordinates(x_dist, y_dist, angle)
        robotiqgrip.close_gripper()
        rob.movejs([high_point, bucket_pos], acc = a, vel = v, radius=0.2)
    #     rob.movej(bucket_pos, acc = a, vel = v)
        robotiqgrip.open_gripper()
    #     rob.movej(pose, acc = a, vel = v)
        rob.movejs([high_point, pose], acc = a, vel = v, radius=0.2)

        angle += 45
    except:
        rob = rob_connect()
    

print("finished everything")

engine.say('Finished searching')
engine.runAndWait()

go_to_start(rob)

engine.say('Program 1 finished')
engine.runAndWait()

rob.close()


In [22]:
## get robot back into init pos
test_robot(a = 1, v = 1)


('robot tcp is at pose: ', [-0.0005396048175256851, -1.6516192595111292, -1.2263291517840784, -1.2765234152423304, -4.722868029271261, 0.0014980281703174114])


Exception in thread Thread-18:
Traceback (most recent call last):
  File "C:\Users\61457\Anaconda3\envs\p2\lib\threading.py", line 801, in __bootstrap_inner
    self.run()
  File "C:\Users\61457\Anaconda3\envs\p2\lib\site-packages\urx-0.11.0-py2.7.egg\urx\ursecmon.py", line 289, in run
    data = self._get_data()
  File "C:\Users\61457\Anaconda3\envs\p2\lib\site-packages\urx-0.11.0-py2.7.egg\urx\ursecmon.py", line 336, in _get_data
    tmp = self._s_secondary.recv(1024)
timeout: timed out

Exception in thread Thread-16:
Traceback (most recent call last):
  File "C:\Users\61457\Anaconda3\envs\p2\lib\threading.py", line 801, in __bootstrap_inner
    self.run()
  File "C:\Users\61457\Anaconda3\envs\p2\lib\site-packages\urx-0.11.0-py2.7.egg\urx\ursecmon.py", line 289, in run
    data = self._get_data()
  File "C:\Users\61457\Anaconda3\envs\p2\lib\site-packages\urx-0.11.0-py2.7.egg\urx\ursecmon.py", line 336, in _get_data
    tmp = self._s_secondary.recv(1024)
timeout: timed out
Exception i

In [ ]:
rob = rob_connect()
robotiqgrip = Robotiq_Two_Finger_Gripper(rob)
a = 0.7; v = 2

pose = rob.getj()
print(pose)
# pose[0] = pose[0] + 0.785398
pose[0] = pose[0] - 6.28319

rob.movej(pose, acc = a, vel = v)

rob.close()

In [2]:
def teach_bins(rob):
    '''
    Allows user to move robot to placement of bins
    Also need to allow user to take photo of object!??
    '''
    colours = []
    bins = []
    n_bins = int(raw_input("Please enter the number of bins you have: "))
    for i in range(n_bins):
        rob.set_freedrive(1, timeout=60)
        colours.append(raw_input("Press color of bin when complete: "))
        bins.append(rob.getj())
        
    print("Completed all bins!")
    return [colours, bins]

def get_image():
    '''
    Recieve latest image and save it in a folder
    '''
    
    image_url = "http://192.168.1.6:4242/current.jpg?type=color"
    
    resp = requests.get(image_url, stream=True)
    # Open a local file with wb ( write binary ) permission.
    img_string = 'images/latest_image.jpg'
    local_file = open(img_string, 'wb')
    # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
    resp.raw.decode_content = True
    # Copy the response stream raw data to local image file.
    shutil.copyfileobj(resp.raw, local_file)
    # Remove the image url response object.
    del resp
    
    im = cv2.imread('images/latest_image.jpg')
    plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
    plt.show()
    
    print("image recieved")

def detect_photos(colour):
    '''
    Read in latest image and detect blobs of the given colour
    return the distance to the top left blob in form [X_from_center, Y_from_center] else returns None
    '''
    f = "images/latest_image.jpg"

    # two ranges because red stretchs over the Hue colour range
    
    ##RED
    lower_red_1 = (0,100,50)
    upper_red_1 = (5,255,255)

    lower_red_2 = (170,100,50)
    upper_red_2 = (180,255,255)
    
    ##GREEN
    lower_green = (45, 100, 50)
    upper_green = (70, 255, 255)
    
    ##YELLOW
    lower_yellow = (30, 100, 50)
    upper_yellow = (45, 255, 255)



    # reading in image to RGB and HSV
    img_RGB = cv2.cvtColor(cv2.imread(f), cv2.COLOR_BGR2RGB)
    img_HSV = cv2.cvtColor(img_RGB, cv2.COLOR_RGB2HSV)

    if colour == 'red':

        # Create two masks
        mask1 = cv2.inRange(img_HSV, lower_red_1, upper_red_1)
        mask2 = cv2.inRange(img_HSV, lower_red_2, upper_red_2)

        # XOR masks
        mask = cv2.bitwise_xor(mask1, mask2)
    elif colour == 'green':
        mask = cv2.inRange(img_HSV, lower_green, upper_green)

    elif colour == 'yellow':
        mask = cv2.inRange(img_HSV, lower_yellow, upper_yellow)


    # bitwise and with image
    result = cv2.bitwise_and(img_HSV, img_HSV, mask=mask)


    # put median blur over the top
    median = cv2.medianBlur(mask,23)

    print("pixel count: " + str(cv2.countNonZero(median)))

    if cv2.countNonZero(median) < 2000:
        print("no more objects in scene of colour: " + colour)
        return None

    # find connected components
    ret, labels = cv2.connectedComponents(median)

    # Map component labels to hue val
    label_hue = np.uint8(179*labels/np.max(labels))
    blank_ch = 255*np.ones_like(label_hue)
    labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])

    # cvt to BGR for display
    labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)

    # set bg label to black
    labeled_img[label_hue==0] = 0

    #########################################################
    ## Finding Centers!
    ret,thresh = cv2.threshold(median,127,255,0)

    # find contours in the binary image
    contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

    distance_min = 100000
    for c in contours:
        # calculate moments for each contour
        M = cv2.moments(c)

        # calculate x,y coordinate of center
        if M["m00"] != 0:
            
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            cv2.circle(labeled_img, (cX, cY), 5, (255, 255, 255), -1)

            distance_new = np.sqrt(cX**2 + cY**2)
            if distance_new < distance_min:
                distance_min = distance_new
                X_top_left = cX
                Y_top_left = cY

            print("centerpoints at: x: " + str(cX) + " y: " + str(cY))

    print("top left points are: x: " + str(X_top_left) + " y: " + str(Y_top_left))

#     X_from_center = X_top_left - (640/2)
#     Y_from_center = Y_top_left - (480/2)

#     print("distance from center: x: " + str(X_from_center) + " y: " + str(Y_from_center))

    # print out figures
    figure = plt.figure(figsize=(20,20))
    plt.subplot(1, 4, 1)
    # mask
    plt.imshow(mask, cmap="gray")
    plt.subplot(1, 4, 2)
    # median blue
    plt.imshow(median)
    #original image
    plt.subplot(1, 4, 3)
    plt.imshow(img_RGB)
    # final 
    plt.subplot(1, 4, 4)
    plt.imshow(labeled_img)

    plt.show()

#     return [X_from_center, Y_from_center]
    return [X_top_left, Y_top_left]

def test_robot(a = 0.2, v = 0.3):
    '''
    Test to see if robot is behaving normally
    '''
    # connecting to robot and initialising gripper
    while(1):
        try:
            rob = urx.Robot("192.168.1.6")
        except:
            print("didnt connect, try again")
            time.sleep(1)
        else:
            break
    
    robotiqgrip = Robotiq_Two_Finger_Gripper(rob)

    robotiqgrip.open_gripper()

    # move to initial position
    rob.movej(cam_init_pos, acc = a, vel = v)


    pose = rob.getj()
    print("robot tcp is at pose: ", pose)

    rob.close()

def go_to_start(rob):
    rob.movej(cam_init_pos, acc = a, vel = v)
    print("moved to init position")
    

def rob_connect():
    while(1):
        try:
            rob = urx.Robot("192.168.1.6")
        except:
            print("didnt connect, try again")
            time.sleep(1)
        else:
            robotiqgrip = Robotiq_Two_Finger_Gripper(rob)
            return rob
    



def convert_to_mm(x, y):
    '''
    return mm from center!
    '''
    x = x - (640/2)
    y = y - (480/2)
    
    
    x = ((44.0/640.0) / 100) * x + 0.05
    x = x/0.8
    y = ((40.0/(480.0)) / 100) * y + 0.04
#     y = y / 1.2
    
    
    return [x, y]

def change_coordinates(x, y, n):
    if n == 0:
        rob.translate((0, x, 0), acc=a, vel=v)
        rob.translate((0, -x, 0), acc=a, vel=v)

        rob.translate((y, 0, 0), acc=a, vel=v)
        rob.translate((-y, 0, 0), acc=a, vel=v)
        
    if n == 45:
#         rob.translate((-math.tan(0.785398) * x, x/math.cos(0.785398), 0), acc=a, vel=v)
#         rob.translate((math.tan(0.785398) * x, -x/math.cos(0.785398), 0), acc=a, vel=v)

#         rob.translate((y/math.cos(0.785398), math.tan(0.785398) * y, 0), acc=a, vel=v)
#         rob.translate((-y/math.cos(0.785398), -math.tan(0.785398) * y, 0), acc=a, vel=v)
        
        rob.translate((-math.tan(0.785398) * x, x/math.cos(0.785398), 0), acc=a, vel=v)
        rob.translate((y/math.cos(0.785398), math.tan(0.785398) * y, 0), acc=a, vel=v)
        rob.translate((-0.02/math.cos(0.785398), math.tan(0.785398) * -0.02, 0), acc=a, vel=v)
        rob.translate((0,0, -0.33), acc=a, vel=v)
        
    if n == 90:
        rob.translate((-x, 0, 0), acc=a, vel=v)
        rob.translate((x, 0, 0), acc=a, vel=v)

        rob.translate((0, y, 0), acc=a, vel=v)
        rob.translate((0, -y, 0), acc=a, vel=v)
        
    if n == 135:
        rob.translate((x/math.cos(0.785398*3), math.tan(0.785398*3) * x, 0), acc=a, vel=v)
        rob.translate((-x/math.cos(0.785398*3), -math.tan(0.785398*3) * x, 0), acc=a, vel=v)

        rob.translate((math.tan(0.785398*3) * y, -y/math.cos(0.785398*3), 0), acc=a, vel=v)
        rob.translate((-math.tan(0.785398*3) * y, y/math.cos(0.785398*3), 0), acc=a, vel=v)
        
    if n == 180:
        rob.translate((0, -x, 0), acc=a, vel=v)
        rob.translate((0, x, 0), acc=a, vel=v)

        rob.translate((-y, 0, 0), acc=a, vel=v)
        rob.translate((y, 0, 0), acc=a, vel=v)
        
    if n == 225:
#         rob.translate((math.tan(0.785398) * x, -x/math.cos(0.785398), 0), acc=a, vel=v)
#         rob.translate((-math.tan(0.785398) * x, y/math.cos(0.785398), 0), acc=a, vel=v)

#         rob.translate((-y/math.cos(0.785398), -math.tan(0.785398) * y, 0), acc=a, vel=v)
#         rob.translate((y/math.cos(0.785398), math.tan(0.785398) * y, 0), acc=a, vel=v)
        
        rob.translate((math.tan(0.785398) * x, -x/math.cos(0.785398), 0), acc=a, vel=v)
        rob.translate((-y/math.cos(0.785398), -math.tan(0.785398) * y, 0), acc=a, vel=v)
        rob.translate((0.07/math.cos(0.785398), math.tan(0.785398) * 0.07, 0), acc=a, vel=v)
        rob.translate((0,0, -0.33), acc=a, vel=v)
        
    if n == 270:
        rob.translate((x, 0, 0), acc=a, vel=v)
        rob.translate((-x, 0, 0), acc=a, vel=v)

        rob.translate((0, -y, 0), acc=a, vel=v)
        rob.translate((0, y, 0), acc=a, vel=v)
        
    if n == 315:
        rob.translate((-x/math.cos(0.785398*3), -math.tan(0.785398*3) * x, 0), acc=a, vel=v)
        rob.translate((x/math.cos(0.785398*3), math.tan(0.785398*3) * x, 0), acc=a, vel=v)

        rob.translate((-math.tan(0.785398*3) * y, y/math.cos(0.785398*3), 0), acc=a, vel=v)
        rob.translate((math.tan(0.785398*3) * y, -y/math.cos(0.785398*3), 0), acc=a, vel=v)


In [21]:
## CALLIBRATING IMAGE

# initalise

%matplotlib tk
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

fig = plt.figure(figsize=(20,30))
get_image()
img=mpimg.imread('images/latest_image.jpg')


all_val = []

def onclick(event):
    ix, iy = event.xdata, event.ydata
    print(ix, iy)
    [x, y] = convert_to_mm(ix, iy)
    print(x, y)
    
    
    rob = rob_connect()
    robotiqgrip = Robotiq_Two_Finger_Gripper(rob)
    a = 0.7; v = 2
    robotiqgrip.close_gripper()
    
    rob.translate((y, x, 0), acc=a, vel=v)
    rob.translate((-0.095, 0, 0), acc=a, vel=v)
    rob.translate((0, 0, -0.33), acc=a, vel=v)
    time.sleep(1)
    rob.movej(cam_init_pos, acc = a, vel = v)
    rob.close()
    

    
cid = fig.canvas.mpl_connect('button_press_event', onclick)

imgplot = plt.imshow(img)
plt.show()

image recieved
(111.00319470911336, 99.94741941166103)
(-0.11710662954685572, -0.07671048382361581)
didnt connect, try again
(111.00319470911336, 99.94741941166103)
(-0.11710662954685572, -0.07671048382361581)


Traceback (most recent call last):
  File "C:\Users\61457\Anaconda3\envs\p2\lib\site-packages\matplotlib\cbook\__init__.py", line 387, in process
    proxy(*args, **kwargs)
  File "C:\Users\61457\Anaconda3\envs\p2\lib\site-packages\matplotlib\cbook\__init__.py", line 227, in __call__
    return mtd(*args, **kwargs)
  File "<ipython-input-21-5dd4abaad788>", line 30, in onclick
    rob.translate((0, 0, -0.33), acc=a, vel=v)
  File "C:\Users\61457\Anaconda3\envs\p2\lib\site-packages\urx-0.11.0-py2.7.egg\urx\urrobot.py", line 477, in translate
    return self.movex(command, p, vel=vel, acc=acc, wait=wait)
  File "C:\Users\61457\Anaconda3\envs\p2\lib\site-packages\urx-0.11.0-py2.7.egg\urx\robot.py", line 180, in movex
    return self.set_pose(t, acc, vel, wait=wait, command=command, threshold=threshold)
  File "C:\Users\61457\Anaconda3\envs\p2\lib\site-packages\urx-0.11.0-py2.7.egg\urx\robot.py", line 105, in set_pose
    pose = URRobot.movex(self, command, t.pose_vector, acc=acc, vel=vel, 

(310.8526670616684, 272.12542636155456)
(0.05463901075612128, 0.0667711886346288)


Traceback (most recent call last):
  File "C:\Users\61457\Anaconda3\envs\p2\lib\site-packages\matplotlib\cbook\__init__.py", line 387, in process
    proxy(*args, **kwargs)
  File "C:\Users\61457\Anaconda3\envs\p2\lib\site-packages\matplotlib\cbook\__init__.py", line 227, in __call__
    return mtd(*args, **kwargs)
  File "<ipython-input-21-5dd4abaad788>", line 30, in onclick
    rob.translate((0, 0, -0.33), acc=a, vel=v)
  File "C:\Users\61457\Anaconda3\envs\p2\lib\site-packages\urx-0.11.0-py2.7.egg\urx\urrobot.py", line 477, in translate
    return self.movex(command, p, vel=vel, acc=acc, wait=wait)
  File "C:\Users\61457\Anaconda3\envs\p2\lib\site-packages\urx-0.11.0-py2.7.egg\urx\robot.py", line 180, in movex
    return self.set_pose(t, acc, vel, wait=wait, command=command, threshold=threshold)
  File "C:\Users\61457\Anaconda3\envs\p2\lib\site-packages\urx-0.11.0-py2.7.egg\urx\robot.py", line 105, in set_pose
    pose = URRobot.movex(self, command, t.pose_vector, acc=acc, vel=vel, 

In [ ]:
#### trying at 90 degrees

rob = rob_connect()
robotiqgrip = Robotiq_Two_Finger_Gripper(rob)
a = 1.5; v = 2
robotiqgrip.open_gripper()
go_to_start(rob)
rob.translate((0, 0.1, 0), acc=a, vel=v)
rob.translate((0, -0.1, 0), acc=a, vel=v)

rob.translate((0.1, 0, 0), acc=a, vel=v)
rob.translate((-0.1, 0, 0), acc=a, vel=v)

pose = rob.getj()
pose[0] = pose[0] + 0.785398
rob.movej(pose, acc = a, vel = v)

# rob.translate((-math.tan(0.785398) * 0.1, 0.1/math.cos(0.785398), 0), acc=a, vel=v)
# rob.translate((math.tan(0.785398) * 0.1, -0.1/math.cos(0.785398), 0), acc=a, vel=v)

# rob.translate((0.1/math.cos(0.785398), math.tan(0.785398) * 0.1, 0), acc=a, vel=v)
# rob.translate((-0.1/math.cos(0.785398), -math.tan(0.785398) * 0.1, 0), acc=a, vel=v)

pose = rob.getj()
pose[0] = pose[0] + 0.785398
rob.movej(pose, acc = a, vel = v)

# rob.translate((-0.1, 0, 0), acc=a, vel=v)
# rob.translate((0.1, 0, 0), acc=a, vel=v)

# rob.translate((0, 0.1, 0), acc=a, vel=v)
# rob.translate((0, -0.1, 0), acc=a, vel=v)

pose = rob.getj()
pose[0] = pose[0] + 0.785398
rob.movej(pose, acc = a, vel = v)

# rob.translate((0.1/math.cos(0.785398*3), math.tan(0.785398*3) * 0.1, 0), acc=a, vel=v)
# rob.translate((-0.1/math.cos(0.785398*3), -math.tan(0.785398*3) * 0.1, 0), acc=a, vel=v)

# rob.translate((math.tan(0.785398*3) * 0.1, -0.1/math.cos(0.785398*3), 0), acc=a, vel=v)
# rob.translate((-math.tan(0.785398*3) * 0.1, 0.1/math.cos(0.785398*3), 0), acc=a, vel=v)

pose = rob.getj()
pose[0] = pose[0] + 0.785398
rob.movej(pose, acc = a, vel = v)


# rob.translate((0, -0.1, 0), acc=a, vel=v)
# rob.translate((0, 0.1, 0), acc=a, vel=v)

# rob.translate((-0.1, 0, 0), acc=a, vel=v)
# rob.translate((0.1, 0, 0), acc=a, vel=v)

pose = rob.getj()
pose[0] = pose[0] + 0.785398
rob.movej(pose, acc = a, vel = v)

rob.translate((math.tan(0.785398) * 0.1, -0.1/math.cos(0.785398), 0), acc=a, vel=v)
rob.translate((-math.tan(0.785398) * 0.1, 0.1/math.cos(0.785398), 0), acc=a, vel=v)

rob.translate((-0.1/math.cos(0.785398), -math.tan(0.785398) * 0.1, 0), acc=a, vel=v)
rob.translate((0.1/math.cos(0.785398), math.tan(0.785398) * 0.1, 0), acc=a, vel=v)

pose = rob.getj()
pose[0] = pose[0] + 0.785398
rob.movej(pose, acc = a, vel = v)

rob.translate((0.1, 0, 0), acc=a, vel=v)
rob.translate((-0.1, 0, 0), acc=a, vel=v)

rob.translate((0, -0.1, 0), acc=a, vel=v)
rob.translate((0, 0.1, 0), acc=a, vel=v)

pose = rob.getj()
pose[0] = pose[0] + 0.785398
rob.movej(pose, acc = a, vel = v)

rob.translate((-0.1/math.cos(0.785398*3), -math.tan(0.785398*3) * 0.1, 0), acc=a, vel=v)
rob.translate((0.1/math.cos(0.785398*3), math.tan(0.785398*3) * 0.1, 0), acc=a, vel=v)

rob.translate((-math.tan(0.785398*3) * 0.1, 0.1/math.cos(0.785398*3), 0), acc=a, vel=v)
rob.translate((math.tan(0.785398*3) * 0.1, -0.1/math.cos(0.785398*3), 0), acc=a, vel=v)


rob.close()